In [ ]:
"""TODO only use these statements if you encounter an import issue (may need to refresh kernel)
import sys
!{sys.executable} -m pip install google-auth google-cloud-vision google-api-python-client google-auth-httplib2 google-auth-oauthlib
"""

# imports for auth and temporary google vision AI test
import requests
import base64
from google.cloud import vision
from google.oauth2 import service_account
from googleapiclient.discovery import build

# TODO: change the path/name to match your local txt file containing your API Key
# ENSURE THE API KEY IS ON THE FIRST LINE IN THE TXT FILE
# Path to the file containing the API key
api_key_file_path = 'path/to/your/api_keys.txt'

# read the API key from the file
def get_api_key_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            if len(lines) >= 2:
                # API key on first line
                return lines[0].strip()
            else:
                raise ValueError("API key not found in the file.")
    except Exception as e:
        print(f"Failed to read the API key from file: {e}")
        return None

# API key for the Google Vision API
API_KEY = get_api_key_from_file(api_key_file_path)

# Google Sheets API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'path/to/your/myJsonFile.json' # TODO PUT YOUR JSON SERVICE ACCOUNT FILE PATH HERE
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=credentials)

# Spreadsheet details below:
# TODO: put the unique spreadsheet ID. Find this between /d/ and /edit in URL
SPREADSHEET_ID = '157Dpz8bSe0NMoUAcElBKG6InbRuiboWqht11-tdlnNA' # PLACEHOLDER ID

READ_RANGE_NAME = 'Sheet1!A1'  # Read Column A, Row 1
WRITE_RANGE_NAME = 'Sheet1!C1'  # Write to Column C, Row 1

# image encoding done below
def encode_image(image_url):
    """Fetches an image from the URL and encodes it in base64."""
    try:
        response = requests.get(image_url)
        response.raise_for_status()  # raise exception for HTTP errors
        return base64.b64encode(response.content).decode()
    except requests.RequestException as e:
        print(f"Failed to fetch image from URL: {e}")
        return None

def detect_text_via_api(image_url):
    url = f"https://vision.googleapis.com/v1/images:annotate?key={API_KEY}"
    headers = {'Content-Type': 'application/json'}
    image_content = encode_image(image_url)
    
    body = {
        "requests": [
            {
                "image": {
                    "content": image_content
                },
                "features": [
                    {
                        "type": "TEXT_DETECTION" # IMPORTANT for Google Vision
                    }
                ]
            }
        ]
    }
    
    # just debugging related
    response = requests.post(url, headers=headers, json=body)
    result = response.json()
    print(result)  # just for debugging, prints API response

    if 'responses' in result and len(result['responses']) > 0 and 'textAnnotations' in result['responses'][0]:
        return result['responses'][0]['textAnnotations'][0]['description']
    else:
        return 'No text detected' 

# calls our prior functions to read image and update the sheet
def read_and_update_sheet():
    # Read the image URL from the sheet
    result = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID, range=READ_RANGE_NAME).execute()
    image_url = result.get('values', [[None]])[0][0]
    if image_url:
        detected_text = detect_text_via_api(image_url)
        # Write what text was detected back to the sheet
        values = [[detected_text]]
        body = {'values': values}
        update_result = service.spreadsheets().values().update(
            spreadsheetId=SPREADSHEET_ID, range=WRITE_RANGE_NAME,
            valueInputOption='USER_ENTERED', body=body).execute()
        print(f'Cells updated: {update_result.get("updatedCells")}')
    else:
        print("No URL found in the specified cell.")

read_and_update_sheet()